In [3]:
#pip install pyfaidx

In [52]:
import pandas as pd
import numpy as np
import torch
import json
import os
from pyfaidx import Fasta
from sklearn.metrics import confusion_matrix
from model import Pangolin
from tqdm import tqdm

# Import hàm của bạn
from metrics import compute_metrics

# --- CẤU HÌNH HỆ THỐNG ---
DATA_FOLDER = r"D:\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val"
RESULTS_FOLDER = "results"
MODEL_FOLDER = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\Pangolin\pretrained_model"
FASTA_PATH = r"D:\Homo_sapiens.GRCh38.dna.primary_assembly.fa"

# Thông số kỹ thuật
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 256
OFFSET = -1      
WINDOW_SIZE = 5
DELTA_MARGIN = 0.000005  # Hiệu số tối thiểu giữa D và A để tin tưởng
BASE_THRESHOLD = 0.9 # Ngưỡng cơ bản

# Kiến trúc Pangolin v3
CONTEXT_WINDOW = 15000
HALF_WINDOW = CONTEXT_WINDOW // 2
N_CHANNELS = 32
W = np.asarray([11, 11, 11, 11, 11, 11, 11, 11, 21, 21, 21, 21, 41, 41, 41, 41])
AR = np.asarray([1, 1, 1, 1, 4, 4, 4, 4, 10, 10, 10, 10, 25, 25, 25, 25])

TEST_FILES = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']
os.makedirs(RESULTS_FOLDER, exist_ok=True)

# --- HÀM HỖ TRỢ ---
genome = Fasta(FASTA_PATH)

def get_sequence(chrom, pos):
    chrom_key = chrom if chrom in genome.keys() else (f"chr{chrom}" if f"chr{chrom}" in genome.keys() else chrom)
    start, end = pos - HALF_WINDOW, pos + HALF_WINDOW
    try:
        return genome[chrom_key][start:end].seq.upper()
    except:
        return "N" * CONTEXT_WINDOW 

# Giữ nguyên hàm mã hóa của bạn
IN_MAP = np.asarray([[0, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
def one_hot_encode_torch(seq, strand):
    seq = seq.upper().replace('A', '1').replace('C', '2').replace('G', '3').replace('T', '4').replace('N', '0')
    if strand == '-':
        seq = np.asarray(list(map(int, list(seq[::-1]))))
        seq = np.where(seq == 0, 0, 5 - seq)
    else:
        seq = np.asarray(list(map(int, list(seq))))
    return IN_MAP[seq.astype('int8')]

# --- KHỞI TẠO ENSEMBLE ---
models = []
WEIGHT_FILES = [f"final.3.{i}.3.v2" for i in range(8)]
for weight_file in WEIGHT_FILES:
    m = Pangolin(L=N_CHANNELS, W=W, AR=AR)
    path = os.path.join(MODEL_FOLDER, weight_file)
    if os.path.exists(path):
        m.load_state_dict(torch.load(path, map_location=DEVICE))
        m.to(DEVICE).eval()
        models.append(m)

# --- VÒNG LẶP CHÍNH ---
for file_name in TEST_FILES:
    input_path = os.path.join(DATA_FOLDER, file_name)
    if not os.path.exists(input_path): continue
        
    print(f"🚀 Benchmarking: {file_name}")
    df = pd.read_csv(input_path)
    y_true = df['Splicing_types'].values
    
    # FIX LỖI 1: Khởi tạo danh sách dự đoán
    probs_list = []
    preds_list = []

    for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Inference"):
        batch_df = df.iloc[i : i + BATCH_SIZE]
        batch_seqs = []
        
        for _, row in batch_df.iterrows():
            parts = row['id'].split('_')
            raw = get_sequence(parts[1], int(parts[2]) + OFFSET)
            encoded = one_hot_encode_torch(raw, parts[3])
            batch_seqs.append(torch.from_numpy(encoded).float().permute(1, 0))
        
        seq_tensor = torch.stack(batch_seqs).to(DEVICE)

        with torch.no_grad():
            # Khởi tạo tensor lưu giá trị MAX qua 8 mô hình (FIX LỖI 4)
            max_d_ensemble = torch.zeros(len(batch_df)).to(DEVICE)
            max_a_ensemble = torch.zeros(len(batch_df)).to(DEVICE)
            
            for m in models:
                output = m(seq_tensor) 
                c_idx, n_t = output.shape[1] // 2, output.shape[-1] // 2
                
                # Sliding Window Max-pooling trong không gian (Window=5)
                half_w = WINDOW_SIZE // 2
                window_out = output[:, c_idx - half_w : c_idx + half_w + 1, :]
                
                # Lấy Max spatial và Max tissue cho từng mô hình
                p_d_m = torch.max(torch.max(window_out[:, :, :n_t], dim=1)[0], dim=1)[0]
                p_a_m = torch.max(torch.max(window_out[:, :, n_t:], dim=1)[0], dim=1)[0]
                
                # FIX LỖI 4: Lấy Max qua các mô hình thay vì Mean
                max_d_ensemble = torch.max(max_d_ensemble, p_d_m)
                max_a_ensemble = torch.max(max_a_ensemble, p_a_m)
            
            d_final = max_d_ensemble.cpu().numpy()
            a_final = max_a_ensemble.cpu().numpy()
            
            # FIX LỖI 2: Xử lý xác suất liên tục và Phân loại
            for d, a in zip(d_final, a_final):
                diff = abs(d - a)
                
                # Tính toán xác suất lớp Neither (0) một cách tự nhiên
                # Nếu d và a thấp, p_neither sẽ cao
                sum_da = d + a
                p_neither = max(0.0001, 1.0 - sum_da)
                
                # Normalize để tổng xác suất bằng 1 (Duy trì AUC chuẩn)
                norm_factor = p_neither + d + a
                prob_vec = [p_neither/norm_factor, a/norm_factor, d/norm_factor] # Đã swap nhãn 1-2

                # Logic phân loại dùng Delta Margin và Threshold
                if diff < DELTA_MARGIN or (d < BASE_THRESHOLD and a < BASE_THRESHOLD):
                    preds_list.append(0)
                    probs_list.append([1.0, 0.0, 0.0]) # Ép nhãn cứng cho Prediction
                else:
                    if d > a:
                        preds_list.append(2) # d (Donor model) -> Acceptor (2)
                        probs_list.append(prob_vec)
                    else:
                        preds_list.append(1) # a (Acceptor model) -> Donor (1)
                        probs_list.append(prob_vec)

    probs = np.array(probs_list)
    y_pred = np.array(preds_list)
    
    res = compute_metrics(y_true, y_pred, probs)
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1, 2])
    
    final = {"metrics": res, "confusion_matrix": cm.tolist()}
    output_path = os.path.join(RESULTS_FOLDER, file_name.replace('.csv', '_results.json'))
    with open(output_path, 'w') as f:
        json.dump(final, f, indent=4)

print("✅ Benchmark hoàn tất!")

🚀 Benchmarking: test_1_1_1.csv


Inference: 100%|██████████| 103/103 [05:25<00:00,  3.16s/it]


🚀 Benchmarking: test_2_1_1.csv


Inference:  22%|██▏       | 31/138 [01:40<05:47,  3.25s/it]


KeyboardInterrupt: 

In [16]:
def diagnostic_check(file_name, num_samples=5):
    df = pd.read_csv(os.path.join(DATA_FOLDER, file_name))
    
    # Lấy mẫu Donor (lớp 1) và Acceptor (lớp 2)
    donor_samples = df[df['Splicing_types'] == 1].head(num_samples)
    acc_samples = df[df['Splicing_types'] == 2].head(num_samples)
    diagnose_df = pd.concat([donor_samples, acc_samples])

    print(f"{'ID (Pos)':<20} | {'Type':<6} | {'Motif (Center)':<15} | {'Avg_D':<8} | {'Avg_A':<8} | {'Pred'}")
    print("-" * 85)

    # Ngưỡng Threshold thực tế để chẩn đoán
    HARD_THRESHOLD = 0.1 

    for _, row in diagnose_df.iterrows():
        parts = row['id'].split('_')
        raw_seq = get_sequence(parts[1], int(parts[2]))
        
        # 1. Kiểm tra Motif tại vị trí trung tâm (7500)
        # Lấy 4nt quanh điểm cắt để dễ quan sát (Vị trí 7498 đến 7502)
        center_motif = raw_seq[HALF_WINDOW-2 : HALF_WINDOW+2] 
        
        # 2. Sửa lỗi AttributeError: Chuyển NumPy sang Tensor trước khi unsqueeze
        # Vì hàm của bạn trả về NumPy array (L, 4), ta cần permute(1, 0) để thành (4, L)
        encoded_seq = one_hot_encode_torch(raw_seq, parts[3])
        seq_tensor = torch.from_numpy(encoded_seq).float().permute(1, 0).unsqueeze(0).to(DEVICE)
        
        sum_d, sum_a = 0, 0
        with torch.no_grad():
            for m in models:
                out = m(seq_tensor) # Output shape: [1, Out_Len, 2*Tissues]
                
                c_idx = out.shape[1] // 2
                n_t = out.shape[-1] // 2
                
                # Tách Donor và Acceptor từ tensor gộp
                p_d = out[0, c_idx, :n_t]
                p_a = out[0, c_idx, n_t:]
                
                sum_d += torch.max(p_d).item()
                sum_a += torch.max(p_a).item()
        
        avg_d, avg_a = sum_d/len(models), sum_a/len(models)
        
        # 3. Áp dụng Threshold thực tế để xác định nhãn dự đoán
        if avg_d < HARD_THRESHOLD and avg_a < HARD_THRESHOLD:
            pred = 0 # Neither
        elif avg_d > avg_a:
            pred = 1 # Donor
        else:
            pred = 2 # Acceptor
            
        print(f"{parts[2]:<20} | {row['Splicing_types']:<6} | {center_motif:^15} | {avg_d:.4f} | {avg_a:.4f} | {pred}")

# Gọi hàm kiểm tra
diagnostic_check(TEST_FILES[0])

ID (Pos)             | Type   | Motif (Center)  | Avg_D    | Avg_A    | Pred
-------------------------------------------------------------------------------------
63733353             | 1      |      GGTG       | 0.9945 | 0.9947 | 2
46150118             | 1      |      ACCA       | 0.9946 | 0.9942 | 1
26842338             | 1      |      ACCT       | 0.9946 | 0.9949 | 2
47786047             | 1      |      ACTC       | 0.9942 | 0.9947 | 2
34017506             | 1      |      GGTA       | 0.9942 | 0.9936 | 1
32743755             | 2      |      CCTA       | 0.9945 | 0.9945 | 2
46137074             | 2      |      TCTG       | 0.9945 | 0.9941 | 1
33819240             | 2      |      AGCA       | 0.9945 | 0.9944 | 1
3035587              | 2      |      AGAA       | 0.9946 | 0.9948 | 2
45455744             | 2      |      AGAG       | 0.9938 | 0.9939 | 2


In [17]:
def find_best_offset(file_name, sample_idx=0):
    df = pd.read_csv(os.path.join(DATA_FOLDER, file_name))
    row = df.iloc[sample_idx]
    parts = row['id'].split('_')
    chrom, pos, strand = parts[1], int(parts[2]), parts[3]
    
    print(f"--- Kiểm tra Offset cho {parts[0]} tại {pos} ({strand}) ---")
    # Kiểm tra trong khoảng offset từ -3 đến +3
    for offset in range(-3, 4):
        actual_pos = pos + offset
        raw_seq = get_sequence(chrom, actual_pos)
        
        # Trích xuất 2nt ngay tại điểm cắt (vị trí 7500-7501)
        # Lưu ý: Với strand '-', ta lấy reverse complement của vùng đó
        center_2nt = raw_seq[HALF_WINDOW : HALF_WINDOW+2]
        
        # Donor chuẩn là GT, Acceptor chuẩn là AG
        status = "MATCH!" if (row['Splicing_types'] == 1 and center_2nt == "GT") or \
                             (row['Splicing_types'] == 2 and center_2nt == "AG") else ""
        
        print(f"Offset {offset:>2}: {center_2nt} {status}")

# Chạy thử để tìm offset đúng
find_best_offset(TEST_FILES[0], sample_idx=0) # Thử với mẫu Donor đầu tiên

--- Kiểm tra Offset cho Neg tại 46105846 (+) ---
Offset -3: AT 
Offset -2: TG 
Offset -1: GC 
Offset  0: CA 
Offset  1: AA 
Offset  2: AA 
Offset  3: AA 


In [20]:
def find_best_offset(file_name, sample_idx=0):
    df = pd.read_csv(os.path.join(DATA_FOLDER, file_name))
    row = df.iloc[sample_idx]
    parts = row['id'].split('_')
    chrom, pos, strand = parts[1], int(parts[2]), parts[3]
    
    print(f"--- Kiểm tra Offset cho {parts[0]} tại {pos} ({strand}) ---")
    # Kiểm tra trong khoảng offset từ -3 đến +3
    for offset in range(-3, 4):
        actual_pos = pos + offset
        raw_seq = get_sequence(chrom, actual_pos)
        
        # Trích xuất 2nt ngay tại điểm cắt (vị trí 7500-7501)
        # Lưu ý: Với strand '-', ta lấy reverse complement của vùng đó
        center_2nt = raw_seq[HALF_WINDOW : HALF_WINDOW+2]
        
        # Donor chuẩn là GT, Acceptor chuẩn là AG
        status = "MATCH!" if (row['Splicing_types'] == 1 and center_2nt == "GT") or \
                             (row['Splicing_types'] == 2 and center_2nt == "AG") else ""
        
        print(f"Offset {offset:>2}: {center_2nt} {status}")

# Chạy thử để tìm offset đúng
find_best_offset(TEST_FILES[0], sample_idx=2) # Thử với mẫu Donor đầu tiên

--- Kiểm tra Offset cho Donor tại 63733353 (+) ---
Offset -3: CG 
Offset -2: GG 
Offset -1: GT MATCH!
Offset  0: TG 
Offset  1: GA 
Offset  2: AG 
Offset  3: GG 


In [21]:
def find_best_offset(file_name, sample_idx=0):
    df = pd.read_csv(os.path.join(DATA_FOLDER, file_name))
    row = df.iloc[sample_idx]
    parts = row['id'].split('_')
    chrom, pos, strand = parts[1], int(parts[2]), parts[3]
    
    print(f"--- Kiểm tra Offset cho {parts[0]} tại {pos} ({strand}) ---")
    # Kiểm tra trong khoảng offset từ -3 đến +3
    for offset in range(-3, 4):
        actual_pos = pos + offset
        raw_seq = get_sequence(chrom, actual_pos)
        
        # Trích xuất 2nt ngay tại điểm cắt (vị trí 7500-7501)
        # Lưu ý: Với strand '-', ta lấy reverse complement của vùng đó
        center_2nt = raw_seq[HALF_WINDOW : HALF_WINDOW+2]
        
        # Donor chuẩn là GT, Acceptor chuẩn là AG
        status = "MATCH!" if (row['Splicing_types'] == 1 and center_2nt == "GT") or \
                             (row['Splicing_types'] == 2 and center_2nt == "AG") else ""
        
        print(f"Offset {offset:>2}: {center_2nt} {status}")

# Chạy thử để tìm offset đúng
find_best_offset(TEST_FILES[0], sample_idx=3) # Thử với mẫu Donor đầu tiên

--- Kiểm tra Offset cho Acc tại 32743755 (-) ---
Offset -3: AC 
Offset -2: CC 
Offset -1: CT 
Offset  0: TA 
Offset  1: AA 
Offset  2: AA 
Offset  3: AA 
